In [23]:
#Import pulp stuff
from pulp import *
from tabulate import tabulate


def Generate_F_Pentominoes(nrows,ncols):
    '''This will return a list of all pentominoes under rotation, reflection, and enlargement. Each element of the list will be a list containing all pentomines of a given size (ordered from smallest size to largest). 
    
    Inputs:
        nrows: Number of rows in the puzzle
        ncols: Number of columns in the puzzle
    Outputs:
        F: A list containing lists representing each of the pentominoes of a given enlargement factor. Each pentomino will be its own list in the list corresponding to the enlargement factor.'''
    
    valmax = min(nrows,ncols)//3
    
    # All rows, columns and values within the puzzle can take values from 0 to 5
    VALS = range(0, valmax+1)
    #There are squares numbered from 1 to 17 in the puzzle grid to be filled
    ROWS = range(1,nrows+1)
    COLS = range(1,ncols+1)

    #Create all of the F pentominoes (We will take the original f shape and have D4 act on it.)

    F = []

    #n gives the current pentomino size
    for n in range(1,valmax+1):
        Fn = []

        #Loop over all of the possible center square locations for a given size
        for i in range(1+n,nrows+1-(2*n-1)):
            for j in range(1+n,ncols+1-(2*n-1)):
                fn = []

                #generate all possible rotations and reflections
                #e
                fn.append([(i+k,j+l) for k in range(n) for l in range(n)] + 
                         [(i-n+k,j+l) for k in range(n) for l in range(n)] +
                         [(i+k,j+l+n) for k in range(n) for l in range(n)] +
                         [(i+k,j+l-n) for k in range(n) for l in range(n)] +
                         [(i+n+k,j+l+n) for k in range(n) for l in range(n)])

                #r
                fn.append([(i+k,j+l) for k in range(n) for l in range(n)] + 
                         [(i-n+k,j+l) for k in range(n) for l in range(n)] +
                         [(i+k,j+l+n) for k in range(n) for l in range(n)] +
                         [(i+k+n,j+l) for k in range(n) for l in range(n)] +
                         [(i+n+k,j+l-n) for k in range(n) for l in range(n)])

                #r^2
                fn.append([(i+k,j+l) for k in range(n) for l in range(n)] + 
                         [(i+n+k,j+l) for k in range(n) for l in range(n)] +
                         [(i+k,j+l+n) for k in range(n) for l in range(n)] +
                         [(i+k,j+l-n) for k in range(n) for l in range(n)] +
                         [(i-n+k,j+l-n) for k in range(n) for l in range(n)])

                #r^3
                fn.append([(i+k,j+l) for k in range(n) for l in range(n)] + 
                         [(i-n+k,j+l) for k in range(n) for l in range(n)] +
                         [(i+k-n,j+l+n) for k in range(n) for l in range(n)] +
                         [(i+k,j+l-n) for k in range(n) for l in range(n)] +
                         [(i+n+k,j+l) for k in range(n) for l in range(n)])

                #f
                fn.append([(i+k,j+l) for k in range(n) for l in range(n)] + 
                         [(i+n+k,j+l) for k in range(n) for l in range(n)] +
                         [(i+k,j+l+n) for k in range(n) for l in range(n)] +
                         [(i+k,j+l-n) for k in range(n) for l in range(n)] +
                         [(i-n+k,j+l+n) for k in range(n) for l in range(n)])

                #fr
                fn.append([(i+k,j+l) for k in range(n) for l in range(n)] + 
                         [(i-n+k,j+l) for k in range(n) for l in range(n)] +
                         [(i+k+n,j+l) for k in range(n) for l in range(n)] +
                         [(i+k,j+l-n) for k in range(n) for l in range(n)] +
                         [(i+n+k,j+l+n) for k in range(n) for l in range(n)])

                #fr^2
                fn.append([(i+k,j+l) for k in range(n) for l in range(n)] + 
                         [(i-n+k,j+l) for k in range(n) for l in range(n)] +
                         [(i+k,j+l+n) for k in range(n) for l in range(n)] +
                         [(i+k,j+l-n) for k in range(n) for l in range(n)] +
                         [(i+n+k,j+l-n) for k in range(n) for l in range(n)])

                #fr^3
                fn.append([(i+k,j+l) for k in range(n) for l in range(n)] + 
                         [(i-n+k,j+l) for k in range(n) for l in range(n)] +
                         [(i+k,j+l+n) for k in range(n) for l in range(n)] +
                         [(i+k+n,j+l) for k in range(n) for l in range(n)] +
                         [(i-n+k,j+l-n) for k in range(n) for l in range(n)])

                Fn += fn

        F.append(Fn)
    
    return F

def Generate_Overlap(nrows, ncols, F):
    '''This generates a list of all of the pentominoes in F that cover a given cell. 
    
    Inputs: 
        nrows: Number of rows in the puzzle
        ncols: Number of columns in the puzzle
        F: The pentominoes from the Generate_Pentominoes function
        
    Outputs:
        overlap: This is a dictionary indexed by tuples (r,c) corresponding to the cell in row r and cell c. For a given key, the corresponding element will be a list containing all of the tuples corresponing to indexed pentominoes in F.'''
    overlap = {}
    for r in range(nrows+1):
        for c in range(ncols+1):
            C = []
            for v in range(len(F)):
                for k in range(len(F[v])):
                    pent = F[v][k]
                    if (r,c) in pent:
                        C.append((v,k))
            overlap[(r,c)] = C
        
        
    return overlap

def SolveProblem(nrows, ncols, rowvals, colvals, regions):
    '''This function will create an IP problem and will solve the given problem. The solution will be printed as a table. 
    
    Inputs:
        nrows: This is the number of rows in the puzzle
        ncols: This is hte number of columns in the puzzle
        rowvals: A dictionary with the row sums. The key is the row, the corresponding value is the sum in the row
        colvals: A dictionary with the column sums. The key is the row, the corresponding value is the sum in the column 
        regions: This is a list containing a lists of tuples. Each internal list will contain a tuple for each of the cells in a given region.
        
    Outputs: None
        '''
    
    #Generate the pentominos
    F = Generate_F_Pentominoes(nrows,ncols)
    
    #Generate the overlap for the pentominoes
    overlap = Generate_Overlap(nrows, ncols, F)
    
    # The prob variable is created to contain the problem data
    prob = LpProblem("Some_F_Squares")

    valmax = min(nrows,ncols)//3

    VALS = range(0, valmax+1)
    ROWS = range(1,nrows+1)
    COLS = range(1,ncols+1)

    numpentominoes = [len(Fn) for Fn in F]
    PENT = [range(x) for x in numpentominoes]


    # Create the Decision Variables
    cellchoices = LpVariable.dicts("Cell Choice", (ROWS, COLS), cat="Integer")
    pentchoices = [LpVariable.dicts("Pentomino Choice " + str(i+1), (PENT[i]), cat="Binary") for i in range(len(PENT))]

    # We do not define an objective function since none is needed. Here we only care about feasibility

    #Allowed cell values
    for r in ROWS:
        for c in COLS:
            prob += 0 <= cellchoices[r][c]
            prob += cellchoices[r][c] <= valmax
    
    #Row and Column Sums
    for r in ROWS:
        try:
            prob += lpSum([cellchoices[r][c] for c in COLS]) == rowvals[r]
        except:
            pass
    
    for c in COLS:
        try:
            prob += lpSum([cellchoices[r][c] for r in ROWS]) == colvals[c]
        except:
            pass
    
    #Create the region sum constraints
    region0 = regions[0]
    for i in range(len(regions)):
        if i == 0:
            continue
        else:
            region = regions[i]
            prob += lpSum(cellchoices[x[0]][x[1]] for x in region) == lpSum(cellchoices[y[0]][y[1]] for y in region0)

    #Create the pentomino overlap constraints and cell equality constraints
    for r in ROWS:
        for c in COLS:
            overs = overlap[(r,c)]
            prob += lpSum(pentchoices[x[0]][x[1]] for x in overs) <= 1
            prob += lpSum((x[0]+1)*pentchoices[x[0]][x[1]] for x in overs) == cellchoices[r][c]


    # The problem data is written to an .lp file
    prob.writeLP("SomeFSquares.lp")

    # The problem is solved using PuLP's choice of Solver
    prob.solve()

    # The status of the solution is printed to the screen
    print("Status:", LpStatus[prob.status])
    
    #Output the solution
    sol = [[0 for i in ROWS] for j in COLS]
    for r in ROWS:
        for c in COLS:
                
                sol[r-1][c-1] = int(value(cellchoices[r][c]))
                
    print(tabulate(sol, tablefmt = "fancy_grid"))
    
    return

# Sample Problem Data #

In [21]:
#This is the data from the solved example problem

nrows0 = 8
ncols0 = 8

rowvals0 = {1:3,
           2:9,
           3:10,
           5:13,
           6:8,
           7:7,
           8:2
              }

colvals0 = {1:9,
            2:12,
            3:12,
            4:10,
            5:8,
            6:8,
            8:2
              }

#These are the regions. All of the contained points are stored as ordered pairs
regions0 = [
    
    #1
    [(1,1),(1,2),(2,1),(3,1)],
    #2
    [(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),(2,4),(2,8),(3,8)],
    #3
    [(2,2),(2,3),(3,2)],
    #4
    [(2,5),(2,6),(2,7),(3,6),(3,7),(4,7),(4,8)],
    #5
    [(3,3),(4,2),(4,3)],
    #6
    [(3,4),(4,4),(5,4)],
    #7
    [(3,5),(4,5),(5,5)],
    #8
    [(4,1),(5,1),(6,1),(7,1)],
    #9
    [(4,6),(5,6),(5,7),(5,8),(6,8),(7,7),(7,8),(8,8)],
    #10
    [(5,2),(5,3),(6,2)],
    #11
    [(6,3),(7,2),(7,3)],
    #12
    [(6,4),(6,5),(6,6),(6,7),(7,5)],
    #13
    [(7,4),(7,6),(8,1),(8,2),(8,3),(8,4),(8,5),(8,6),(8,7)]
]

SolveProblem(nrows0, ncols0, rowvals0, colvals0, regions0)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.8/dist-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/2e131a8b64eb4942b6f102e8e916dc13-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/2e131a8b64eb4942b6f102e8e916dc13-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 287 COLUMNS
At line 7309 RHS
At line 7592 BOUNDS
At line 8018 ENDATA
Problem MODEL has 282 rows, 425 columns and 6172 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.01 seconds
Cgl0003I 41 fixed, 1 tightened bounds, 22 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 2 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 32 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 28 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 19 strength

# Problem Data #

In [25]:
#This is the data for the monthly puzzle

nrows = 17
ncols = 17

rowvals = {1:14,
           2:24,
           3:24,
           4:39,
           5:43,
           7:22,
           8:23,
           9:29,
           10:28,
           11:34,
           12:36,
           13:29,
           14:26,
           15:26,
           16:24,
           17:20
              }

colvals = {1:13,
           2:20,
           3:22,
           4:28,
           5:30,
           6:36,
           7:35,
           8:39,
           9:49,
           10:39,
           11:39,
           13:23,
           14:32,
           15:23,
           16:17,
           17:13
              }

#These are the regions. All of the contained points are stored as ordered pairs
regions = [
    
    #1
    {(1,1),(1,2),(2,1),(2,2),(2,3),(3,1),(4,1),(5,1),(6,1),(7,1),(8,1),(9,1),(10,1),(11,1),(12,1),(13,1),(14,1),(15,1),(15,5),(15,6),(16,1),(16,2),(16,5),(17,1),(17,2),(17,3),(17,4),(17,5),(17,6)},
    #2
    {(1,3),(1,4),(2,4),(2,5),(2,6),(2,7),(3,5),(3,6),(3,7),(4,4),(4,5),(4,6),(4,7),(5,4)},
    #3
    {(1,5),(1,6),(1,7),(1,8),(1,9),(1,10),(2,8),(2,9),(2,10),(3,8),(3,10),(4,10)},
    #4
    {(1,11),(1,12),(1,13),(1,14),(1,15),(1,16),(1,17),(2,11),(2,16),(2,17),(3,11),(3,13),(3,14),(3,17),(4,11),(4,12),(4,13),(4,17),(5,12),(5,13),(5,17),(6,12),(6,13),(7,12),(7,13),(8,12)},
    #5
    {(2,12),(2,13),(2,14),(2,15),(3,12),(3,15),(4,14),(4,15),(5,14),(6,14),(6,15),(7,15)},
    #6
    {(3,2),(3,3),(3,4),(4,2),(4,3),(5,2),(6,2),(7,2),(8,2),(9,2),(10,2),(11,2),(12,2),(13,2),(14,2),(15,2),(15,3),(16,3),(16,4)},
    #7
    {(3,9),(4,8),(4,9),(5,5),(5,6),(5,7),(5,8),(6,5)},
    #8
    {(3,16),(4,16),(5,15),(5,16),(6,16),(6,17),(7,16),(7,17),(8,15),(8,16),(8,17),(9,16),(9,17),(10,15),(10,16),(10,17),(11,17),(12,15),(12,17),(13,15),(13,16),(13,17),(14,16),(14,17),(15,17),(16,15),(16,16),(16,17),(17,15),(17,16),(17,17)},
    #9
    {(5,3),(6,3),(6,4),(6,6),(7,4),(7,5),(7,6),(8,5)},
    #10
    {(5,9),(6,7),(6,8),(6,9),(7,7),(7,8),(7,9),(8,6),(8,7),(9,6),(9,7)},
    #11
    {(5,10),(5,11),(6,11),(7,11),(8,11),(8,13),(9,9),(9,10),(9,11),(9,12),(9,13),(10,11),(10,13)},
    #12
    {(6,10),(7,10),(8,8),(8,9),(8,10),(9,8),(10,7),(10,8),(11,6),(11,7),(12,5),(12,6)},
    #13
    {(7,3),(8,3),(8,4),(9,3),(9,4),(9,5),(10,3),(10,4),(10,5),(10,6),(11,3),(11,4),(11,5),(12,3),(12,4),(13,3)},
    #14
    {(7,14),(8,14),(9,14),(9,15),(10,12),(10,14),(11,12),(11,13),(11,14),(11,15),(11,16),(12,16)},
    #15
    {(10,9),(10,10),(11,10),(11,11),(12,11),(12,12),(13,12),(13,13)},
    #16
    {(11,8),(11,9),(12,7),(12,8),(12,9),(13,4),(13,5),(13,6),(13,7),(13,8),(14,3),(14,4),(14,5),(14,6),(14,7),(14,8),(15,4)},
    #17
    {(12,10),(13,9),(13,10),(13,11),(14,9),(14,11),(15,8),(15,9)},
    #18
    {(12,13),(12,14),(13,14),(14,12),(14,13),(14,14),(14,15),(15,11),(15,12),(15,15),(15,16)},
    #19
    {(14,10),(15,10),(15,13),(15,14),(16,8),(16,9),(16,10),(16,11),(16,12),(16,13),(16,14),(17,14)},
    #20
    {(15,7),(16,6),(16,7),(17,7),(17,8),(17,9),(17,10),(17,11),(17,12),(17,13)}
    
]

SolveProblem(nrows, ncols, rowvals, colvals, regions)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/lib/python3.8/dist-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/417abd3985cc4ab8a5538b5baac91610-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/417abd3985cc4ab8a5538b5baac91610-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1212 COLUMNS
At line 198414 RHS
At line 199622 BOUNDS
At line 203873 ENDATA
Problem MODEL has 1207 rows, 4250 columns and 188702 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 3.18 seconds
Cgl0003I 11 fixed, 0 tightened bounds, 15 strengthened rows, 0 substitutions
Cgl0003I 3 fixed, 0 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 3 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 12 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1